In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

2024-09-17 16:03:02.008996: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 16:03:02.031155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 16:03:02.363991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#定义所有的目录和文件夹
data_dir = r'/home/liuyq/data/ar_data'
os.getcwd()
os.chdir('/mnt/sda/liuyq/asso_rule2')
os.getcwd()

'/mnt/sda/liuyq/asso_rule2'

In [5]:
from utils.data_process.load_matrix_data import TransformData

In [29]:
# 读取数据
data_path = os.path.join(data_dir, 'sc_data', 'pbmc')
# 导入LoadMtrixData类
from utils.data_process.load_data import LoadMatrixDataReal, TransformDataReal, read_cell_type
from utils.algorithms.association_rule import AssociationRule


In [31]:
data_obj = LoadMatrixDataReal(data_path)
_data = data_obj.load_data()

In [32]:
_data.shape

(2700, 32738)

In [33]:
#过滤支持度小于0.1的基因
_data = data_obj.filter_genes(_data, threshold=0.1)

In [34]:
_data.shape

(2700, 2107)

In [35]:
#读取cell type
cell_type_path = os.path.join(data_path, 'cell_type.tsv')
cell_type = read_cell_type(cell_type_path)

In [38]:
print("number of data:", len(cell_type))
print("unique cell_type:", set(cell_type))

number of data: 2700
unique cell_type: {'FCGR3A+_Mono', 'B', 'DC', 'CD14+_Mono', 'Memory_CD4_T', 'Naive_CD4_T', 'Platelet', 'NK', 'CD8_T'}


In [40]:
#基于cell_type进行数据的划分
adata_subs = data_obj.split_by_cell_type(_data, cell_type)
transform_data = {}

/mnt/sda/liuyq/asso_rule2/utils/data_process/load_data.py:69: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cell_type'] = cell_type


In [41]:
len(adata_subs)

9

In [42]:
for cell_type, adata in adata_subs.items():
    adata = data_obj.filter_genes(adata)
    print(adata)
    transform_data_obj = TransformDataReal(adata)
    data, genes_info, cells_info = transform_data_obj.run()
    transform_data[cell_type] = {'data': data, 'genes_info': genes_info, 'cells_info': cells_info}

View of AnnData object with n_obs × n_vars = 684 × 1659
    obs: 'cell_type'
    var: 'gene_ids'


2024-09-16 17:41:45.410198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-16 17:41:45.412928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-16 17:41:45.413058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

稀疏矩阵大小： tf.Tensor(1134756, shape=(), dtype=int32)
float32大小： tf.Tensor(1134756, shape=(), dtype=int32)
load data success
View of AnnData object with n_obs × n_vars = 350 × 1485
    obs: 'cell_type'
    var: 'gene_ids'
稀疏矩阵大小： tf.Tensor(519750, shape=(), dtype=int32)
float32大小： tf.Tensor(519750, shape=(), dtype=int32)
load data success
View of AnnData object with n_obs × n_vars = 427 × 1927
    obs: 'cell_type'
    var: 'gene_ids'
稀疏矩阵大小： tf.Tensor(822829, shape=(), dtype=int32)
float32大小： tf.Tensor(822829, shape=(), dtype=int32)
load data success
View of AnnData object with n_obs × n_vars = 492 × 1698
    obs: 'cell_type'
    var: 'gene_ids'
稀疏矩阵大小： tf.Tensor(835416, shape=(), dtype=int32)
float32大小： tf.Tensor(835416, shape=(), dtype=int32)
load data success
View of AnnData object with n_obs × n_vars = 154 × 1898
    obs: 'cell_type'
    var: 'gene_ids'
稀疏矩阵大小： tf.Tensor(292292, shape=(), dtype=int32)
float32大小： tf.Tensor(292292, shape=(), dtype=int32)
load data success
View of AnnData

In [43]:
#对每一个细胞类型进行关联规则的挖掘
from utils.algorithms.association_rule import AssociationRule
from utils.algorithms.matrix_association_rule import MatrixRule

In [44]:
data_tem_ct, data_dic = transform_data.popitem()

In [45]:
data_tem_ct

'Platelet'

In [46]:
data_dic

{'data': <tf.Tensor: shape=(893, 13), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 1., 0., 1.],
        [0., 1., 0., ..., 1., 0., 0.],
        [1., 1., 0., ..., 1., 1., 1.]], dtype=float32)>,
 'genes_info': Index(['ISG15', 'AURKAIP1', 'MRPL20', 'SSU72', 'GNB1', 'TNFRSF14', 'RPL22',
        'PARK7', 'ENO1', 'PGD',
        ...
        'PRMT2', 'MT-ND1', 'MT-ND2', 'MT-CO1', 'MT-CO2', 'MT-ATP6', 'MT-CO3',
        'MT-ND4', 'MT-ND5', 'MT-CYB'],
       dtype='object', length=893),
 'cells_info': Index(['ACCCACTGGTTCAG-1', 'ACGAACTGGCTATG-1', 'AGTCTTACTTCGGA-1',
        'ATCATCTGACACCA-1', 'ATTCAGCTCATTGG-1', 'GACGCTCTCTCTCG-1',
        'GAGTTGTGGTAGCT-1', 'GCGCATCTGGTTAC-1', 'GGAACACTTCAGAC-1',
        'GGCATATGGGGAGT-1', 'GTCATACTTCGCCT-1', 'TAACACCTTGTTTC-1',
        'TTACGTACGTTCAG-1'],
       dtype='object')}

In [47]:
matrix_rule = MatrixRule(data_dic['data'], data_dic['genes_info'], data_dic['cells_info'])

In [49]:
data = matrix_rule.all_metrics_to_python

TypeError: 'dict' object is not callable

In [50]:
data

{'support': array([[0.15384616, 0.15384616, 0.07692308, ..., 0.15384616, 0.07692308,
         0.15384616],
        [0.15384616, 0.23076923, 0.15384616, ..., 0.23076923, 0.15384616,
         0.23076923],
        [0.07692308, 0.15384616, 0.15384616, ..., 0.15384616, 0.15384616,
         0.15384616],
        ...,
        [0.15384616, 0.23076923, 0.15384616, ..., 0.53846157, 0.30769232,
         0.46153846],
        [0.07692308, 0.15384616, 0.15384616, ..., 0.30769232, 0.30769232,
         0.30769232],
        [0.15384616, 0.23076923, 0.15384616, ..., 0.46153846, 0.30769232,
         0.84615386]], dtype=float32),
 'confidence': array([[1.        , 0.6666667 , 0.5       , ..., 0.28571427, 0.25      ,
         0.18181819],
        [1.        , 1.        , 1.        , ..., 0.4285714 , 0.5       ,
         0.27272728],
        [0.5       , 0.6666667 , 1.        , ..., 0.28571427, 0.5       ,
         0.18181819],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        , 1.    

In [51]:
result_pd = matrix_rule.all_metrics_to_dataframe

In [52]:
result_pd

,metric,ISG15,AURKAIP1,MRPL20,SSU72,GNB1,TNFRSF14,RPL22,PARK7,ENO1,...,PRMT2,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
ISG15,support,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.153846,...,0.000000,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.153846,0.076923,0.153846
AURKAIP1,support,0.153846,0.230769,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.230769,...,0.076923,0.230769,0.230769,0.230769,0.230769,0.153846,0.230769,0.230769,0.153846,0.230769
MRPL20,support,0.076923,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.153846,...,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846
SSU72,support,0.076923,0.153846,0.153846,0.307692,0.153846,0.076923,0.153846,0.153846,0.153846,...,0.153846,0.230769,0.307692,0.307692,0.230769,0.307692,0.307692,0.153846,0.153846,0.307692
GNB1,support,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.153846,0.153846,0.153846,...,0.076923,0.230769,0.230769,0.230769,0.153846,0.153846,0.230769,0.153846,0.153846,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ATP6,conviction,0.461538,0.692308,inf,inf,0.692308,inf,inf,0.692308,1.153846,...,inf,0.846154,1.038462,0.923077,2.076923,inf,0.923077,0.807692,inf,2.538462
MT-CO3,conviction,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,0.692308,0.769231,inf,inf,inf,0.846154
MT-ND4,conviction,inf,inf,inf,0.923077,1.384615,0.923077,inf,0.923077,inf,...,1.384615,1.269231,1.038461,1.107692,1.384615,0.923077,1.107692,inf,inf,1.015385
MT-ND5,conviction,1.384615,2.076923,inf,1.384615,2.076923,1.384615,inf,1.038462,1.730769,...,2.076923,1.087912,1.246154,1.038462,1.246154,1.153846,1.038462,1.615385,inf,1.087912


In [112]:
from utils.data_process.ar_metrics_process import SaveArMetrics, LoadArMetrics, FilterArMetrics

In [58]:
result_pds = {'pbmc': result_pd}

In [67]:
threshold_dict={'support': 0.1, 'confidence': 0.6,}

In [68]:
results_filter = FilterArMetrics(result_pds, threshold_dict)

In [69]:
res1 = results_filter.filter_result()

In [70]:
res1

{'pbmc':               metric     ISG15  AURKAIP1    MRPL20     SSU72      GNB1  \
 ISG15        support  0.153846  0.153846       NaN       NaN       NaN   
 AURKAIP1     support  0.153846  0.230769  0.153846  0.153846  0.153846   
 MRPL20       support       NaN  0.153846  0.153846  0.153846  0.153846   
 SSU72        support       NaN  0.153846  0.153846  0.307692  0.153846   
 GNB1         support       NaN  0.153846  0.153846  0.153846  0.230769   
 ...              ...       ...       ...       ...       ...       ...   
 MT-ATP6   conviction  0.461538  0.692308       inf       inf  0.692308   
 MT-CO3    conviction       inf       inf       inf       inf       inf   
 MT-ND4    conviction       inf       inf       inf  0.923077  1.384615   
 MT-ND5    conviction  1.384615  2.076923       inf  1.384615  2.076923   
 MT-CYB    conviction       inf       inf       inf       inf  0.461538   
 
           TNFRSF14     RPL22     PARK7      ENO1  ...     PRMT2    MT-ND1  \
 ISG15      

In [72]:
result_pds['pbmc'].head()

,metric,ISG15,AURKAIP1,MRPL20,SSU72,GNB1,TNFRSF14,RPL22,PARK7,ENO1,...,PRMT2,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
ISG15,support,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.153846,...,0.000000,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.153846,0.076923,0.153846
AURKAIP1,support,0.153846,0.230769,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.230769,...,0.076923,0.230769,0.230769,0.230769,0.230769,0.153846,0.230769,0.230769,0.153846,0.230769
MRPL20,support,0.076923,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.153846,...,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846
SSU72,support,0.076923,0.153846,0.153846,0.307692,0.153846,0.076923,0.153846,0.153846,0.153846,...,0.153846,0.230769,0.307692,0.307692,0.230769,0.307692,0.307692,0.153846,0.153846,0.307692
GNB1,support,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.153846,0.153846,0.153846,...,0.076923,0.230769,0.230769,0.230769,0.153846,0.153846,0.230769,0.153846,0.153846,0.153846


In [75]:
# 创建数据
import pandas as pd
data = {
    'A': [1, 2, 3, 4],
    'B': [5, 6, 7, 8]
}

# 创建双重索引
index = pd.MultiIndex.from_tuples(
    [('foo', 'one'), ('foo', 'two'), ('bar', 'one'), ('bar', 'two')],
    names=['first', 'second']
)

# 创建DataFrame
df = pd.DataFrame(data, index=index)

print(df)

              A  B
first second      
foo   one     1  5
      two     2  6
bar   one     3  7
      two     4  8


In [76]:
df['level_0'] = ['foo', 'foo', 'bar', 'bar']
df['level_1'] = ['one', 'two', 'one', 'two']


In [77]:
df

A  B level_0 level_1
first second                      
foo   one     1  5     foo     one
      two     2  6     foo     two
bar   one     3  7     bar     one
      two     4  8     bar     two

In [81]:
# 设置双重索引
df.set_index(["A"], inplace=True)

KeyError: "None of ['A'] are in the columns"

In [82]:
df

,B
A,
1,5
2,6
3,7
4,8


In [86]:
from copy import deepcopy


In [93]:
result_pds2 = deepcopy(result_pds['pbmc'])

In [94]:
result_pds2

,metric,ISG15,AURKAIP1,MRPL20,SSU72,GNB1,TNFRSF14,RPL22,PARK7,ENO1,...,PRMT2,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
ISG15,support,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.153846,...,0.000000,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.153846,0.076923,0.153846
AURKAIP1,support,0.153846,0.230769,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.230769,...,0.076923,0.230769,0.230769,0.230769,0.230769,0.153846,0.230769,0.230769,0.153846,0.230769
MRPL20,support,0.076923,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.153846,...,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846
SSU72,support,0.076923,0.153846,0.153846,0.307692,0.153846,0.076923,0.153846,0.153846,0.153846,...,0.153846,0.230769,0.307692,0.307692,0.230769,0.307692,0.307692,0.153846,0.153846,0.307692
GNB1,support,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.153846,0.153846,0.153846,...,0.076923,0.230769,0.230769,0.230769,0.153846,0.153846,0.230769,0.153846,0.153846,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ATP6,conviction,0.461538,0.692308,inf,inf,0.692308,inf,inf,0.692308,1.153846,...,inf,0.846154,1.038462,0.923077,2.076923,inf,0.923077,0.807692,inf,2.538462
MT-CO3,conviction,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,0.692308,0.769231,inf,inf,inf,0.846154
MT-ND4,conviction,inf,inf,inf,0.923077,1.384615,0.923077,inf,0.923077,inf,...,1.384615,1.269231,1.038461,1.107692,1.384615,0.923077,1.107692,inf,inf,1.015385
MT-ND5,conviction,1.384615,2.076923,inf,1.384615,2.076923,1.384615,inf,1.038462,1.730769,...,2.076923,1.087912,1.246154,1.038462,1.246154,1.153846,1.038462,1.615385,inf,1.087912


In [104]:
result_pds2.set_index(["metric"], append=True, inplace=True)

In [105]:
result_pds2

,,ISG15,AURKAIP1,MRPL20,SSU72,GNB1,TNFRSF14,RPL22,PARK7,ENO1,PGD,...,PRMT2,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
,metric,,,,,,,,,,,,,,,,,,,,,
ISG15,support,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,0.153846,0.000000,...,0.000000,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.153846,0.076923,0.153846
AURKAIP1,support,0.153846,0.230769,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.230769,0.000000,...,0.076923,0.230769,0.230769,0.230769,0.230769,0.153846,0.230769,0.230769,0.153846,0.230769
MRPL20,support,0.076923,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,0.153846,0.000000,...,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846
SSU72,support,0.076923,0.153846,0.153846,0.307692,0.153846,0.076923,0.153846,0.153846,0.153846,0.000000,...,0.153846,0.230769,0.307692,0.307692,0.230769,0.307692,0.307692,0.153846,0.153846,0.307692
GNB1,support,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.153846,0.153846,0.153846,0.076923,...,0.076923,0.230769,0.230769,0.230769,0.153846,0.153846,0.230769,0.153846,0.153846,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ATP6,conviction,0.461538,0.692308,inf,inf,0.692308,inf,inf,0.692308,1.153846,0.230769,...,inf,0.846154,1.038462,0.923077,2.076923,inf,0.923077,0.807692,inf,2.538462
MT-CO3,conviction,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,0.692308,0.769231,inf,inf,inf,0.846154
MT-ND4,conviction,inf,inf,inf,0.923077,1.384615,0.923077,inf,0.923077,inf,0.923077,...,1.384615,1.269231,1.038461,1.107692,1.384615,0.923077,1.107692,inf,inf,1.015385


In [96]:
result_pds2.reset_index(inplace=True)

,index,metric,ISG15,AURKAIP1,MRPL20,SSU72,GNB1,TNFRSF14,RPL22,PARK7,...,PRMT2,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB
0,ISG15,support,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.076923,0.000000,...,0.000000,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.153846,0.076923,0.153846
1,AURKAIP1,support,0.153846,0.230769,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,...,0.076923,0.230769,0.230769,0.230769,0.230769,0.153846,0.230769,0.230769,0.153846,0.230769
2,MRPL20,support,0.076923,0.153846,0.153846,0.153846,0.153846,0.076923,0.153846,0.076923,...,0.076923,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846,0.153846
3,SSU72,support,0.076923,0.153846,0.153846,0.307692,0.153846,0.076923,0.153846,0.153846,...,0.153846,0.230769,0.307692,0.307692,0.230769,0.307692,0.307692,0.153846,0.153846,0.307692
4,GNB1,support,0.076923,0.153846,0.153846,0.153846,0.230769,0.076923,0.153846,0.153846,...,0.076923,0.230769,0.230769,0.230769,0.153846,0.153846,0.230769,0.153846,0.153846,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,MT-ATP6,conviction,0.461538,0.692308,inf,inf,0.692308,inf,inf,0.692308,...,inf,0.846154,1.038462,0.923077,2.076923,inf,0.923077,0.807692,inf,2.538462
4461,MT-CO3,conviction,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,0.692308,0.769231,inf,inf,inf,0.846154
4462,MT-ND4,conviction,inf,inf,inf,0.923077,1.384615,0.923077,inf,0.923077,...,1.384615,1.269231,1.038461,1.107692,1.384615,0.923077,1.107692,inf,inf,1.015385
4463,MT-ND5,conviction,1.384615,2.076923,inf,1.384615,2.076923,1.384615,inf,1.038462,...,2.076923,1.087912,1.246154,1.038462,1.246154,1.153846,1.038462,1.615385,inf,1.087912


In [106]:
result_pds2.index

MultiIndex([(   'ISG15',    'support'),
            ('AURKAIP1',    'support'),
            (  'MRPL20',    'support'),
            (   'SSU72',    'support'),
            (    'GNB1',    'support'),
            ('TNFRSF14',    'support'),
            (   'RPL22',    'support'),
            (   'PARK7',    'support'),
            (    'ENO1',    'support'),
            (     'PGD',    'support'),
            ...
            (   'PRMT2', 'conviction'),
            (  'MT-ND1', 'conviction'),
            (  'MT-ND2', 'conviction'),
            (  'MT-CO1', 'conviction'),
            (  'MT-CO2', 'conviction'),
            ( 'MT-ATP6', 'conviction'),
            (  'MT-CO3', 'conviction'),
            (  'MT-ND4', 'conviction'),
            (  'MT-ND5', 'conviction'),
            (  'MT-CYB', 'conviction')],
           names=[None, 'metric'], length=4465)

In [109]:
a = result_pds2.loc["ISG15", "AURKAIP1"]

In [110]:
type(a)

pandas.core.series.Series

In [113]:
type(a)

pandas.core.series.Series

In [233]:
from utils.data_process.ar_metrics_process import SaveArMetrics, LoadArMetrics, FilterArMetrics

In [235]:
results_filter = FilterArMetrics(result_pds, threshold_dict)

In [236]:
new_df = results_filter.transform_to_pairs_in_df(result_pds2)

In [237]:
new_df

metric,antecedent,consequent,confidence,conviction,leverage,lift,support
0,ABI1,ISG15,0.000000,0.846154,-0.023669,0.000000,0.000000
1,ABI1,AURKAIP1,0.333333,1.269231,0.041420,2.166667,0.076923
2,ABI1,MRPL20,0.500000,1.692308,0.053254,3.250000,0.076923
3,ABI1,SSU72,0.250000,1.128205,0.029586,1.625000,0.076923
4,ABI1,GNB1,0.333333,1.269231,0.041420,2.166667,0.076923
...,...,...,...,...,...,...,...
797444,ZYX,MT-ATP6,0.300000,0.989011,-0.005917,0.975000,0.230769
797445,ZYX,MT-CO3,0.333333,1.038462,0.023669,1.083333,0.307692
797446,ZYX,MT-ND4,0.285714,0.969231,-0.011834,0.928571,0.153846
797447,ZYX,MT-ND5,0.250000,0.923077,-0.017751,0.812500,0.076923


In [239]:
import importlib

import utils.data_process.ar_metrics_process

In [238]:
importlib.reload(utils.data_process.ar_metrics_process)

<module 'utils.data_process.ar_metrics_process' from '/mnt/sda/liuyq/asso_rule2/utils/data_process/ar_metrics_process.py'>

In [266]:
from utils.pipe_analysis.subnet_detection import Graph

In [217]:
new_df

metric,antecedent,consequent,confidence,conviction,leverage,lift,support
0,ABI1,ISG15,0.000000,0.846154,-0.023669,0.000000,0.000000
1,ABI1,AURKAIP1,0.333333,1.269231,0.041420,2.166667,0.076923
2,ABI1,MRPL20,0.500000,1.692308,0.053254,3.250000,0.076923
3,ABI1,SSU72,0.250000,1.128205,0.029586,1.625000,0.076923
4,ABI1,GNB1,0.333333,1.269231,0.041420,2.166667,0.076923
...,...,...,...,...,...,...,...
797444,ZYX,MT-ATP6,0.300000,0.989011,-0.005917,0.975000,0.230769
797445,ZYX,MT-CO3,0.333333,1.038462,0.023669,1.083333,0.307692
797446,ZYX,MT-ND4,0.285714,0.969231,-0.011834,0.928571,0.153846
797447,ZYX,MT-ND5,0.250000,0.923077,-0.017751,0.812500,0.076923


In [241]:
new_df_dic = {'pbmc': new_df}

In [242]:
res_edge_list = results_filter.export_edge_list(new_df_dic)


In [243]:
res_edge_list

{'pbmc': metric antecedent consequent
 0            ABI1      ISG15
 1            ABI1   AURKAIP1
 2            ABI1     MRPL20
 3            ABI1      SSU72
 4            ABI1       GNB1
 ...           ...        ...
 797444        ZYX    MT-ATP6
 797445        ZYX     MT-CO3
 797446        ZYX     MT-ND4
 797447        ZYX     MT-ND5
 797448        ZYX     MT-CYB
 
 [796556 rows x 2 columns]}

In [265]:
importlib.reload(utils.pipe_analysis.subnet_detection)

<module 'utils.pipe_analysis.subnet_detection' from '/mnt/sda/liuyq/asso_rule2/utils/pipe_analysis/subnet_detection.py'>

In [8]:
g = Graph(res_edge_list['pbmc'])

NameError: name 'Graph' is not defined

In [256]:
res_edge_list['pbmc'].shape

(796556, 2)

In [261]:
res_edge_list['pbmc'].iloc[0, 2]

IndexError: index 2 is out of bounds for axis 0 with size 2

In [270]:
communities = g.community_detection()

In [272]:
set(communities.values())

{0}

In [273]:
communities

{'ABI1': 0,
 'ISG15': 0,
 'AURKAIP1': 0,
 'MRPL20': 0,
 'SSU72': 0,
 'GNB1': 0,
 'TNFRSF14': 0,
 'RPL22': 0,
 'PARK7': 0,
 'ENO1': 0,
 'PGD': 0,
 'AGTRAP': 0,
 'CAPZB': 0,
 'MINOS1': 0,
 'CDC42': 0,
 'RPL11': 0,
 'SRRM1': 0,
 'SYF2': 0,
 'TMEM50A': 0,
 'LDLRAP1': 0,
 'SH3BGRL3': 0,
 'CD52': 0,
 'NUDC': 0,
 'WASF2': 0,
 'DNAJC8': 0,
 'ATPIF1': 0,
 'TAF12': 0,
 'LAPTM5': 0,
 'PTP4A2': 0,
 'MARCKSL1': 0,
 'PSMB2': 0,
 'AKIRIN1': 0,
 'NDUFS5': 0,
 'CAP1': 0,
 'PPT1': 0,
 'PPCS': 0,
 'YBX1': 0,
 'RPS8': 0,
 'NASP': 0,
 'UQCRH': 0,
 'CMPK1': 0,
 'TXNDC12': 0,
 'SCP2': 0,
 'TMEM59': 0,
 'JAK1': 0,
 'IFI44L': 0,
 'GNG5': 0,
 'SH3GLB1': 0,
 'SEP15': 0,
 'RPL5': 0,
 'DR1': 0,
 'GNAI3': 0,
 'LAMTOR5': 0,
 'RAP1A': 0,
 'CAPZA1': 0,
 'RHOC': 0,
 'ATP1A1': 0,
 'TXNIP': 0,
 'MRPS21': 0,
 'ENSA': 0,
 'CTSS': 0,
 'CDC42SE1': 0,
 'S100A10': 0,
 'S100A9': 0,
 'S100A8': 0,
 'S100A6': 0,
 'S100A4': 0,
 'JTB': 0,
 'RPS27': 0,
 'TPM3': 0,
 'C1orf43': 0,
 'FDPS': 0,
 'SSR2': 0,
 'LAMTOR2': 0,
 'APOA1BP': 0,
 

In [274]:
import networkx as nx

In [278]:
# 节点数
num_nodes = g.number_of_nodes()
# 边数
num_edges = g.number_of_edges()
# 平均度数
avg_degree = sum(dict(g.degree()).values()) / float(num_nodes)

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Average degree: {avg_degree}")

AttributeError: 'Graph' object has no attribute 'number_of_nodes'

In [284]:
G = g.G

In [285]:
G.number_of_edges()

398278

In [294]:
nx.summary(G)

AttributeError: module 'networkx' has no attribute 'summary'

In [288]:
G.number_of_edges()

398278

In [293]:
G.number_of_nodes()


893

In [291]:
communities = g.community_detection()

In [292]:
communities

{'ABI1': 0,
 'ISG15': 0,
 'AURKAIP1': 0,
 'MRPL20': 0,
 'SSU72': 0,
 'GNB1': 0,
 'TNFRSF14': 0,
 'RPL22': 0,
 'PARK7': 0,
 'ENO1': 0,
 'PGD': 0,
 'AGTRAP': 0,
 'CAPZB': 0,
 'MINOS1': 0,
 'CDC42': 0,
 'RPL11': 0,
 'SRRM1': 0,
 'SYF2': 0,
 'TMEM50A': 0,
 'LDLRAP1': 0,
 'SH3BGRL3': 0,
 'CD52': 0,
 'NUDC': 0,
 'WASF2': 0,
 'DNAJC8': 0,
 'ATPIF1': 0,
 'TAF12': 0,
 'LAPTM5': 0,
 'PTP4A2': 0,
 'MARCKSL1': 0,
 'PSMB2': 0,
 'AKIRIN1': 0,
 'NDUFS5': 0,
 'CAP1': 0,
 'PPT1': 0,
 'PPCS': 0,
 'YBX1': 0,
 'RPS8': 0,
 'NASP': 0,
 'UQCRH': 0,
 'CMPK1': 0,
 'TXNDC12': 0,
 'SCP2': 0,
 'TMEM59': 0,
 'JAK1': 0,
 'IFI44L': 0,
 'GNG5': 0,
 'SH3GLB1': 0,
 'SEP15': 0,
 'RPL5': 0,
 'DR1': 0,
 'GNAI3': 0,
 'LAMTOR5': 0,
 'RAP1A': 0,
 'CAPZA1': 0,
 'RHOC': 0,
 'ATP1A1': 0,
 'TXNIP': 0,
 'MRPS21': 0,
 'ENSA': 0,
 'CTSS': 0,
 'CDC42SE1': 0,
 'S100A10': 0,
 'S100A9': 0,
 'S100A8': 0,
 'S100A6': 0,
 'S100A4': 0,
 'JTB': 0,
 'RPS27': 0,
 'TPM3': 0,
 'C1orf43': 0,
 'FDPS': 0,
 'SSR2': 0,
 'LAMTOR2': 0,
 'APOA1BP': 0,
 

In [295]:
a = ["safe", "dsafe", "fdsafewfe"]

In [296]:
a.extend(["ff", "ss", "dd"])

In [297]:
a

['safe', 'dsafe', 'fdsafewfe', 'ff', 'ss', 'dd']

In [6]:
from utils.data_process.load_data import LoadMatrixDataReal, TransformDataReal

In [7]:
G

NameError: name 'G' is not defined

In [9]:
import networkx as nx

# 创建一个简单的图
G = nx.Graph()
G.add_edge(1, 2)
G.add_edge(2, 3)
G.add_edge(3, 4)


In [ ]:
# 保存图对象到Pickle文件
nx.write_gpickle(G, "/mnt/data/graph.pkl")

In [10]:
nx.write_gpickle(G, "/home/liuyq/data/ar_result/ar/graph.pkl")

AttributeError: module 'networkx' has no attribute 'write_gpickle'

In [11]:
nx.__version__

'3.0'

In [19]:
import pickle
from copy import deepcopy

In [20]:
b = {"a":G, "b":deepcopy(G)}

In [21]:
fp = "/home/liuyq/data/ar_result/ar/graph.pkl"

In [25]:
with open(fp, 'wb') as f:
    pickle.dump(b, f)

In [15]:
G

In [26]:
with open(fp, 'rb') as f:
    g_new = pickle.load(f)

In [27]:
g_new

{'a': <networkx.classes.graph.Graph at 0x757c3f224f40>,
 'b': <networkx.classes.graph.Graph at 0x757c3f227eb0>}

In [28]:
#build a pandas dataframe
import pandas as pd
import numpy as np

# 创建一个简单的DataFrame对象
data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'City': ['New York', 'Los Angeles', 'Chicago']
}

df = pd.DataFrame(data)
df
    



,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [35]:
len(G.nodes)

4

In [36]:
len(G.edges)

3

,Unnamed: 0,Name,Age,City
0,0,Alice,25,New York
1,1,Bob,30,Los Angeles
2,2,Charlie,35,Chicago
